In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import math as mt
import random as rng
from sklearn import preprocessing

Load the data from the last step

In [2]:
data = np.load("grouped.npy", allow_pickle=True)
target = np.load("groupedTarg.npy", allow_pickle=True)

Perform the train and validation split here. The data is split before scaling so that the validation data is scaled after the fitting on the train data- since we are assuming the scenario where a predictive maintenance model will need to handle the scenario of dealing with new data and calling a standardise tranform long after the fitting has taken place. The split is conducted randomly so that 1- this script can be re-run to change the data in the train/test split, 2- any outliers are hopefully spread across both splits. 


The normaliser doesn't remove columns which never change value unlike the standardiser which does this automatically through pca

In [3]:
data = data.tolist()
target = target.tolist()

val = []
valTarget = []

valAmount = mt.floor(len(target)/5)
trainAmount = len(target) - valAmount


for i in range(mt.floor(len(target)/5)):
    index = rng.randint(0, len(target)-1)
    val.append(data.pop(index))
    valTarget.append(target.pop(index))

train = data
trainTarget = target


Flatten the data so it can be standardised

In [4]:
def flatten(array):
    flat_array = []
    
    for group in array:
        for i in group:
            flat_array.append(i)
    return flat_array


trainFlat = flatten(train)
trainTargetFlat = flatten(trainTarget)
valFlat = flatten(val)
valTargetFlat = flatten(valTarget)

In [5]:
print(len(trainFlat))
print(len(valFlat))

12018
3006


Create the normaliser so  the tranformation can be done. Fit the preprocessors to the training data then perform a tranform on the train and validation data

In [6]:
from sklearn import preprocessing
scaler = preprocessing.MinMaxScaler()

scaledTrain = scaler.fit_transform(np.array(trainFlat))
scaledVal = scaler.transform(np.array(valFlat))

Regroup the data before saving it again.  The data is now ready for the neural network notebook

In [7]:
def regroup(data, target, amount):
    grouped = []
    groupedTarg = []
    
    for i in range(amount):
        grouped.append([])
        groupedTarg.append([])
    
    total = 0
    for i in range(len(grouped)):
        j = total
        if i!= len(grouped)-1:
            while target[j] > target[j+1]:
                grouped[i].append(data[j])
                groupedTarg[i].append(target[j])
                j+=1
            grouped[i].append(data[j])
            groupedTarg[i].append(target[j])
            total = j + 1

    for i in range(total, len(target)):
        grouped[len(grouped)-1].append(data[i])
        groupedTarg[len(grouped)-1].append(target[i])
    
    return grouped, groupedTarg

In [8]:
train, targetTrain = regroup(scaledTrain.tolist(), trainTargetFlat, trainAmount)
val, valTargetTrain = regroup(scaledVal.tolist(), valTargetFlat, valAmount)



np.save("finished/normTrain", np.array(train))
np.save("finished/normTrainTarget", np.array(targetTrain))

np.save("finished/normVal", np.array(val))
np.save("finished/normValTarget", np.array(valTargetTrain))